# Reproject Sentinel-2 data to WGS84

In [1]:
import rasterio
from rasterio.merge import merge
from rasterio.plot import show
from rasterio.warp import calculate_default_transform, reproject, Resampling
import glob
import os

In [2]:
def reproject_raster(in_path, out_path, dst_crs = 'EPSG:4326'):
    # reproject raster to project crs
    with rasterio.open(in_path) as src:
        src_crs = src.crs
        transform, width, height = calculate_default_transform(src_crs, dst_crs, src.width, src.height, *src.bounds)
        kwargs = src.meta.copy()

        kwargs.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height})

        with rasterio.open(out_path, 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest)
    return(out_path)

In [3]:
# File and folder paths
dirpath = r"data\sentinel2"
search_criteria = "*\R10m\TCI.jp2"
q = os.path.join(dirpath, search_criteria)
print(q)

data\sentinel2\*\R10m\TCI.jp2


In [4]:
paths = glob.glob(q)
paths

['data\\sentinel2\\31PFP,2021-06-14,0\\R10m\\TCI.jp2',
 'data\\sentinel2\\31PFQ,2021-06-14,0\\R10m\\TCI.jp2',
 'data\\sentinel2\\31PFR,2021-06-14,0\\R10m\\TCI.jp2',
 'data\\sentinel2\\31PGP,2021-06-14,0\\R10m\\TCI.jp2',
 'data\\sentinel2\\31PGQ,2021-06-14,0\\R10m\\TCI.jp2',
 'data\\sentinel2\\31PGR,2021-06-14,0\\R10m\\TCI.jp2',
 'data\\sentinel2\\31PHP,2021-06-14,0\\R10m\\TCI.jp2',
 'data\\sentinel2\\31PHQ,2021-06-14,0\\R10m\\TCI.jp2',
 'data\\sentinel2\\31PHR,2021-06-14,0\\R10m\\TCI.jp2',
 'data\\sentinel2\\31TGM,2021-02-23,0\\R10m\\TCI.jp2']

In [5]:
files_to_mosaic = []

In [6]:
for path_in in paths:
    folders_split = path_in.split('\\')
    filename_split = folders_split[-1].split('.')
    filename_split[0] = 'transformed_tci'
    
    filename_out = '.'.join(filename_split)
    folders_split[-1] = filename_out
    path_out = '\\'.join(folders_split)
    
    reproject_raster(path_in, path_out)